In [1]:
import modern_robotics as mr
import numpy as np
from closed_loop_utils import four_bar, two_arm_parallel, stewart_platform, rpr_3dof, jacobian_numeric, is_singular

4 bar links:

- Ground link (L1)

Fixed base between the two ground pivots.

In setup: a horizontal bar of length L1.

- Input crank (L2)

The link you drive with an actuator (input angle = theta1).

Rotates about the left ground pivot.

- Coupler (L3)

The “floating” link between the crank and the rocker.

Connects the crank tip to the rocker tip.

Angle = theta2 in the solver.

- Output rocker (L4)

The output link, pivoted at the right ground pivot.

Angle = theta3 in the solver.
  

In [2]:
# Link lengths
L1,L2,L3,L4 = 1,1,1,1

# Input crank angle
theta1 = np.pi/6

# Solve closure
theta2, theta3 = four_bar(theta1,L1,L2,L3,L4)
print("Four-bar angles (closure solver):", theta2, theta3)

# Compute end-effector positions geometrically
# Crank tip
x_crank = L2 * np.cos(theta1)
y_crank = L2 * np.sin(theta1)

# Coupler tip (where coupler meets rocker)
x_coupler = x_crank + L3 * np.cos(theta2)
y_coupler = y_crank + L3 * np.sin(theta2)

# Rocker tip (should coincide with coupler tip)
x_rocker = L1 + L4 * np.cos(theta3)
y_rocker = L4 * np.sin(theta3)

print("Coupler end position:", (x_coupler, y_coupler))
print("Rocker end position:", (x_rocker, y_rocker))
print("Closure error:", np.hypot(x_coupler-x_rocker, y_coupler-y_rocker))


Four-bar angles (closure solver): -3.638612601411797e-15 0.523598775598296
Coupler end position: (1.8660254037844388, 0.4999999999999963)
Rocker end position: (1.8660254037844402, 0.49999999999999756)
Closure error: 1.8452761694870047e-15


In [6]:
L = 1.0  # link length
xd, yd = 1.0, 0.5  # target grasp point

# Solve closure with your function
th1, th2, th3, th4 = two_arm_parallel(xd, yd, L)
print("Two-arm joint angles (closure solver):")
print("  Arm1:", th1, th2)
print("  Arm2:", th3, th4)

# -----------------------------
# Geometric FK (for validation)
# -----------------------------
x1 = L*np.cos(th1) + L*np.cos(th1+th2)
y1 = L*np.sin(th1) + L*np.sin(th1+th2)
x2 = 2*L + L*np.cos(th3) + L*np.cos(th3+th4)
y2 = L*np.sin(th3) + L*np.sin(th3+th4)

print("End-effector pos Arm1:", (x1, y1))
print("End-effector pos Arm2:", (x2, y2))
print("Target grasp point:   ", (xd, yd))
print("Closure error Arm1:", np.hypot(x1-xd, y1-yd))
print("Closure error Arm2:", np.hypot(x2-xd, y2-yd))

# -----------------------------
# FK with modern_robotics (Arm1)
# -----------------------------
# Home configuration: EE at (2L,0)
M1 = np.array([[1,0,0,2*L],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])

# Screw axes for Arm1 (in space frame, base at (0,0))
Slist1 = np.array([[0,0,1,0,0,0],   # joint1 at (0,0)
                   [0,0,1,0,-L,0]]).T  # joint2 at (L,0)

T1 = mr.FKinSpace(M1, Slist1, [th1, th2])
print("FK with mr (Arm1):", T1[:2, 3])

# -----------------------------
# FK with modern_robotics (Arm2)
# -----------------------------
# Home configuration: EE at (d+2L, 0) where d=2L
d = 2*L
M2 = np.array([[1,0,0,d+2*L],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])

# Screw axes for Arm2 (must account for base at (d,0))
# Joint1: rotation about z through (d,0)
# Joint2: rotation about z through (d+L,0)
Slist2 = np.array([[0,0,1,0,-d,0],        # joint1
                   [0,0,1,0,-(d+L),0]]).T # joint2

T2 = mr.FKinSpace(M2, Slist2, [th3, th4])
print("FK with mr (Arm2):", T2[:2, 3])

Two-arm joint angles (closure solver):
  Arm1: 1.441244159644398 -1.9551931012916544
  Arm2: 3.6555415952342947 -8.23837840847062
End-effector pos Arm1: (1.000000000000288, 0.49999999999734973)
End-effector pos Arm2: (1.0000000000004732, 0.5000000000000245)
Target grasp point:    (1.0, 0.5)
Closure error Arm1: 2.6658703107241357e-12
Closure error Arm2: 4.738127640536351e-13
FK with mr (Arm1): [1.  0.5]
FK with mr (Arm2): [1.  0.5]
